# HW11: 歌曲种类分类

如果有问题请在群里讨论，题意问题请@姜腾

ddl周六（4.16）23:59，提交前请自己重启kernel运行一遍，只提交ipynb文件，不要把数据打包

由于大家期中临近，所以这次作业很大一部分已经由助教给出。尤其是对英文语料预处理的部分，只是希望大家注意一下处理英文语料和中文语料的差别之处，比如需要Lemmatization（比如把过去式还原），还有去除停词。

基本任务描述：
本数据集是已经整理拼接好的数据集，包含了r&b, latin, rock, pop, rap, edm六个音乐种类的歌曲的歌词和音频特征。

特征部分是用的Spotify的api提供的音频分析数据，已经由原音频文件特征工程得来（如果大家对音频分析感兴趣，可以去搜一下[librosa](http://librosa.org/doc/latest/index.html)包，是专业分析音频的包。12个音频特征如下: acousticness, danceability, durationms, energy, instrumentalness, key, liveness, loudness, mode, speechiness, tempo, valence.具体含义参考[链接](https://developer.spotify.com/web-api/get-audio-features/)，比如key的1和0就是大小调的区别。

为了任务简单，本次只用rock, pop, rap三种的音乐，每种抽样1500首歌。预测目标是playlist_genre，即歌曲所属的歌单表征歌曲的种类。

In [3]:
# 可能用到的包
import pandas as pd
import numpy as np
import random

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
# 也有可能用到其他的方法，比如什么Complement Naive Bayes，如果你觉得有必要的话

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

random.seed(1)
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sebastianjiang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# 数据读入和预处理

nRowsRead = None # specify 'None' if want to read whole file
data = pd.read_csv('./spotify_songs.csv', delimiter=',', nrows = nRowsRead)

# data preprocessing

# step1: Removing Numbers, Punctuations and Lowercasing the Words¶

def rid_of_specials(lyrics):
    return re.sub('[^A-Za-z]+', ' ', lyrics).lower()
data["lyrics"] = data["lyrics"].astype(str).apply(rid_of_specials)


# step2: remove stopwords

sw_nltk = (stopwords.words('english'))
stop_words = set(sw_nltk)

def remove_sw(x):
    x = x.split(' ')
    return  ' '.join(z for z in x if z not in stop_words)
stopped = data["lyrics"].apply(remove_sw)

# step3: lemmatizing
lemmatizer = WordNetLemmatizer()
lemmatized = [lemmatizer.lemmatize(i) for i in stopped]
prepeared_sentence = [''.join(j) for j in lemmatized]
data['Lyrics_Processed'] = prepeared_sentence

In [5]:
data.columns

Index(['track_id', 'track_name', 'track_artist', 'lyrics', 'track_popularity',
       'track_album_id', 'track_album_name', 'track_album_release_date',
       'playlist_name', 'playlist_id', 'playlist_genre', 'playlist_subgenre',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_ms', 'language', 'Lyrics_Processed'],
      dtype='object')

In [6]:
data['playlist_genre'].value_counts()

pop      3993
rock     3521
rap      3391
r&b      3326
latin    2178
edm      2045
Name: playlist_genre, dtype: int64

In [7]:
data = data[data['language']=='en'] # 只选出英文歌
# 只留下pop，rock和rap
data = data[data['playlist_genre']!='latin']
data = data[data['playlist_genre']!='edm']
data = data[data['playlist_genre']!='r&b']
data = data.groupby('playlist_genre').sample(n=1500) #每种乐曲选出1500首

### 任务1: 只使用歌词

歌词已经处理好了，任务为**只用歌词**实现朴素贝叶斯分类（注意一下使用哪种Naive Bayes）和Logistic回归分类。将歌词按照课上讲过的方法通过Count Vectorization或者TF-IDF Vectorization向量化作为特征输入训练模型，最后打印在测试集上的classification_report和confusion matrix。

提示：强烈建议直接调包，会很简单的。可能用到的方法已经在最前面给出了。
如果你遇到了TFIDF Vector能不能用于Multinomial Bayes的问题，这一篇blog或许可以解答你的疑惑。https://stackoverflow.com/questions/43237286/how-can-we-use-tfidf-vectors-with-multinomial-naive-bayes

In [1]:
# todo

### 任务2: 接下来考虑加入音频特征，只使用朴素贝叶斯方法。流程如下：

1. 接着上一问，使用训练集训练的朴素贝叶斯模型去在整个数据集做预测，得到三个种类的probability，将其中的两个（因为三者加和为1）作为两个新的特征和'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness','acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo','duration_ms'这些音频特征中你觉得对预测有用的特征拼接在一起，作为新的特征（如果你觉得有必要的话，可以对特征进行进一步处理）；
2. 使用新的特征，选择合适的朴素贝叶斯分类器，再次进行训练；
3. 最后打印在测试集上的classification_report和confusion matrix；
4. 你觉得这么操作有意义吗？请说明。

提示：强烈建议直接调包，会很简单的。model.predict_proba(X)可以调出模型给出的分为各类的预测概率。关于最后一步，有时候理论正确的方法不一定有好的效果，有好的效果的方法不一定理论正确，看你自己的理解。

In [2]:
# todo

### 附加题（2分）：

你有没有其他能增大预测精确率和召回率的想法？叙述你的想法的原理和操作流程（1'），并简单用代码实现（1'）.设计时进行如下思考：
1. （如果要使用深度学习模型）思考NLP任务中one-hot带来的维度灾难对深度学习模型训练带来的挑战，以及词向量化的解决方法
2. 思考如何用恰当的方法，巧妙，优雅而有效地结合使用多种数据（在这个背景下就是文本数据和音频特征的结合使用）

祝大家期中顺利～

In [2]:
# todo